In [36]:
from utils import Utils 
import pandas as pd
utils=Utils()

# BASE PLAYER_SCRAPER

In [37]:
df = utils.cargar_excel("Mundial_Historic_players.xlsx")
df['Año'].unique()

array([1966, 1970, 1974, 1986, 1994, 1998, 2002, 2006, 2014, 2018])

In [ ]:
year=1994
df_filtrado=utils.filtrar_dataframe(df,'Año',year)

In [39]:
df_links_names=utils.buscar_wikipedia_jugadores(df_filtrado)

c:\Users\ser_s\Desktop\WorldCup\futbol_players_scrapper\utils.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["wiki_link"] = wiki_links


## TO EXCEL

In [40]:
utils.guardar_excel(df_links_names,f'LINK_PLAYERS_{year}')

Archivo guardado como: LINK_PLAYERS_2006.xlsx
